In [42]:
%%capture
%run ../img2vec/img2vec.py

In [43]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import base64
import io
from PIL import Image 

In [44]:
SERVER_IP = "213.136.94.240"
USERNAME = "admin"
PASSWORD = "04061997"

client = MongoClient(host=SERVER_IP, username=USERNAME, password=PASSWORD)

In [45]:
db = client.instagramAuditor

In [46]:
profiles = db.profiles

In [97]:
test_profiles = list(profiles.find().limit(2))

In [76]:
username = 'username'
fullname = 'fullname'
biography = 'biography'
followed_by = 'followedby'
follow = 'follow'
is_business_account = 'isbusinessaccount'
is_joined_recently = 'isjoinedrecently'
business_category_name = 'businesscategoryname'
business_category_name = 'categoryid'
is_private = 'isprivate'
is_verified = 'isverified'
has_connected_fb = 'connectedfbpage'
profile_pic = 'profilepic'
profile_pic_url = 'profilepicurl'
posts_count = 'postscount'
posts = 'posts'
image = 'image'
profile_pic_embedding = 'profilepicembedding'
image_embedding = 'profilepicembedding'

In [61]:
def to_pil(raw):
    image_buf = io.BytesIO()
    base64.decode(io.StringIO(raw), image_buf)
    return Image.open(image_buf)

In [64]:
def extract_post_pics(profile):
    return [to_pil(post[image]) for post in profile[posts]]

In [68]:
img2vec = Image2Vec()

In [98]:
test_profiles

[{'_id': ObjectId('5ea2811237fdd072e4bb2b85'),
  'id': '1000037738',
  'username': 'shmkristina',
  'fullname': '𝕶𝖗𝖎𝖘𝖙𝖎𝖓𝖆 𝕳𝖆𝖗𝖔𝖓𝖔𝖛𝖆',
  'biography': '',
  'followedby': 304,
  'follow': 742,
  'isbusinessaccount': False,
  'isjoinedrecently': False,
  'businesscategoryname': '',
  'categoryid': '',
  'isprivate': True,
  'isverified': False,
  'connectedfbpage': False,
  'profilepicurl': 'https://scontent-iad3-1.cdninstagram.com/v/t51.2885-19/s150x150/84151410_2828453183908290_7850661954606596096_n.jpg?_nc_ht=scontent-iad3-1.cdninstagram.com&_nc_ohc=vnfjt8GgRcMAX9qn_Vk&oh=bfdacd9fb69b4687bb47c54bb6d5a088&oe=5ECA7478',
  'profilepic': '/9j/4AAQSkZJRgABAgAAAQABAAD/7QCEUGhvdG9zaG9wIDMuMAA4QklNBAQAAAAAAGccAigAYkZCTUQwMTAwMGFhODAzMDAwMDkwMDUwMDAwOGQwODAwMDAyMzA5MDAwMGNjMDkwMDAwMGIwZTAwMDAyNzEzMDAwMGE0MTMwMDAwNWExNDAwMDAyMzE1MDAwMDBjMWQwMDAwAP/iAhxJQ0NfUFJPRklMRQABAQAAAgxsY21zAhAAAG1udHJSR0IgWFlaIAfcAAEAGQADACkAOWFjc3BBUFBMAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAD21gABAAAAANMtbGNtcwAAAAAAAAAAAAAAAA

In [67]:
img2vec.apply_batch(extract_post_pics(test_profiles[1]))

array([[0.702834  , 1.0936033 , 1.0842735 , ..., 2.6496718 , 0.96915394,
        0.9603366 ],
       [2.5851693 , 2.121459  , 2.1079803 , ..., 0.76422733, 1.3362845 ,
        0.6449115 ],
       [0.93216527, 2.15238   , 0.14577645, ..., 1.5172557 , 0.6650831 ,
        0.8392149 ],
       ...,
       [0.16259244, 0.12807433, 0.61226314, ..., 0.23484705, 1.6888697 ,
        0.06656094],
       [0.83014166, 0.22925362, 0.6103896 , ..., 1.0137109 , 0.56307024,
        0.4426976 ],
       [0.06053875, 1.2545116 , 0.38969558, ..., 0.41352996, 0.9360528 ,
        0.09559195]], dtype=float32)

In [93]:
def add_embeddings(img2vec, profile):
    if profile_pic in profile:
        profile[profile_pic_embedding] = img2vec.apply_single(to_pil(profile[profile_pic]))
    
    if posts in profile and len(profile[posts]) > 0:
        for post in profile[posts]:
            if image in post:
                post[image_embedding] = img2vec.apply_single(to_pil(post[image]))

In [81]:
add_embeddings(img2vec, test_profiles[0])

In [94]:
add_embeddings(img2vec, test_profiles[1])